# Segmenting and Clustering Neighborhoods in Toronto

## Step 1: Scraping and Structuring the Data

In [38]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
from bs4 import BeautifulSoup
import json
from pandas.io.json import json_normalize

In [47]:
# since Wikipedia is cencored in my country, I had to use the wikizero link for the same information.
# I manually copied the page source to file postal_codes.html and parsed from there.
# url_for_postal_codes = 'https://en.0wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# response = requests.get(url_for_postal_codes)

with open('postal_codes.html') as f:
    page_content = f.read()

In [49]:
from collections import OrderedDict

soup = BeautifulSoup(page_content, 'lxml')

# get the part where the table is defined
table = soup.find('table')

# read header
header = table.find('tr').text.strip().split("\n")

# create empty dataframe
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

processed_data = OrderedDict()

# visit all table rows excluding the header
for row in table.find_all('tr')[1:]:
    code, borough, neigh = row.text.strip().split("\n")
    
    if borough != 'Not assigned':
        if neigh == 'Not assigned':
            neigh = borough
    
        if code in processed_data:
            processed_data[code][1] += ", %s" % neigh
        else:
            processed_data.update({code: [borough, neigh]})
        
for pcode, value in processed_data.items():
    df = df.append({'PostalCode': pcode, 
                    'Borough': value[0], 
                    'Neighborhood': value[1]}, ignore_index=True)

In [51]:
df.shape

(103, 3)

## Step 2: Getting the latitude and the longitude coordinates of each neighborhood

In [77]:
lat_long_df = pd.read_csv('Geospatial_Coordinates.csv')
lat_long_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [78]:
lat_long_df.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
lat_long_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [79]:
df = df.merge(lat_long_df, on='PostalCode')
df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
